# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.dataset import Dataset
import joblib
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import LocalWebservice, AciWebservice
import json
import numpy as np
from azureml.automl.core.shared import constants

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'myexperiments'

experiment=Experiment(ws, experiment_name)

#Environment(name="Auto-ML")

In [3]:
cpu_cluster_name = "auto-ml"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Cluster Found!.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
#data_url = 'https://raw.githubusercontent.com/Harini-Pavithra/Machine-Learning-Engineer-with-Microsoft-Azure-Nanodegree/main/Capstone%20Project/Dataset/Heart_Failure_Clinical_Records_Dataset.csv'
#ds = TabularDatasetFactory.from_delimited_files(path=data_url)
#df = ds.to_pandas_dataframe()
#x = df.drop(columns=['DEATH_EVENT'])
#y = df['DEATH_EVENT']
#x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.33, random_state=42)
#train_data=x.join(y)

dataset = Dataset.get_by_name(ws, name='Heart_Failure_Clinical_Records_Dataset')
Heart_Failure_Clinical_Records_Dataset = dataset.to_pandas_dataframe()

Heart_Failure_Clinical_Records_Dataset.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "task":'classification',
    "primary_metric":'accuracy',
    "training_data":dataset,
    "label_column_name":'DEATH_EVENT',
    "n_cross_validations":5,
    "max_concurrent_iterations": 4,
    "featurization": 'auto'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=cpu_cluster,
    **automl_settings)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

{'runId': 'AutoML_049437f4-dc2d-42cb-bed0-5250e552d910',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2021-01-18T13:22:04.336257Z',
 'endTimeUtc': '2021-01-18T14:02:42.365047Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'auto-ml',
  'AMLSettingsJsonString': '{"path":null,"name":"myexperiments","subscription_id":"f9d5a085-54dc-4215-9ba6-dad5d86e60a0","resource_group":"aml-quickstarts-135212","workspace_name":"quick-starts-ws-135212","region":"southcentralus","compute_target":"auto-ml","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_min":null,"y_max":null,"num_classes":null,"featurization

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [8]:
remote_run.get_metrics('accuracy')

{'accuracy': 0.8763276836158193}

In [9]:
print(remote_run.get_portal_url())

https://ml.azure.com/experiments/myexperiments/runs/AutoML_049437f4-dc2d-42cb-bed0-5250e552d910?wsid=/subscriptions/f9d5a085-54dc-4215-9ba6-dad5d86e60a0/resourcegroups/aml-quickstarts-135212/workspaces/quick-starts-ws-135212


In [10]:
best_run, best_model = remote_run.get_output()

In [11]:
print(best_run)

Run(Experiment: myexperiments,
Id: AutoML_049437f4-dc2d-42cb-bed0-5250e552d910_106,
Type: azureml.scriptrun,
Status: Completed)


In [12]:
print(best_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                        n_estimators=400,
                                                                                                        n_iter_no_change=None,
                                                                                                        presort='deprecated',
                 

In [13]:
print(best_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('45',
                                           Pipeline(memory=None,
                                                    steps=[('sparsenormalizer',
                                                            <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7fb81b015ef0>),
                                                           ('xgboostclassifier',
                                                            XGBoostClassifier(base_score=0.5,
                                                                              booster='gbtree',
                                                                              colsample_bylevel=1,
                                                                              colsample_bynode=1,
                                                                              colsample_bytree=1,
                               

In [14]:
#TODO: Save the best model
joblib.dump(value=best_model, filename='output/best-automl.pkl')

['output/best-automl.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
#register the model

#model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'best-automl.pkl')
#model = Model.register(workspace=ws,model_path=model_path,model_name='best-automl')
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
#best_run.download_file()
#model.download()
#print(model.name, model.id, model.version, sep='\t')

model = Model.register(workspace=ws,model_name = "best-automl", model_path = './output/best-automl.pkl')
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
print(model.name, model.id, model.version, sep='\t')

Registering model best-automl
best-automl	best-automl:1	1


In [16]:
#create an inference config and deploy the model as a web service
#env = Environment.from_conda_specification(name="myenv",file_path="myenv.yml")
#inferencefinal = InferenceConfig(entry_script='score.py',environment=env) 
#deployment_config = AciWebservice.deploy_configuration()
#service = Model.deploy(workspace=ws,name='deploy', models=[model], inference_config=inferencefinal, deployment_config=deployment_config)
#service.wait_for_deployment(show_output = True)


env = Environment.from_conda_specification('myenv', 'myenv.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True,tags = {'name':'failure Prediction'},description='Heart failure prediction model')
service = model.deploy(workspace=ws,name="automl-deploy",models=[model],inference_config=inference_config,deployment_config=aci_config)

service.wait_for_deployment(show_output=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [17]:
service.update(enable_app_insights=True)

In [18]:
print(service.state)

Healthy


In [19]:
print(service.scoring_uri)

http://d53633fa-66f5-4514-b6d7-ca93810d841b.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
row = 0
x = Heart_Failure_Clinical_Records_Dataset.iloc[row, :-1].to_dict()
y = Heart_Failure_Clinical_Records_Dataset.iloc[row, -1]



In [21]:
print(x)

{'age': 75.0, 'anaemia': 0.0, 'creatinine_phosphokinase': 582.0, 'diabetes': 0.0, 'ejection_fraction': 20.0, 'high_blood_pressure': 1.0, 'platelets': 265000.0, 'serum_creatinine': 1.9, 'serum_sodium': 130.0, 'sex': 1.0, 'smoking': 0.0, 'time': 4.0}


In [22]:
print(y)

1


In [23]:
#input_data = json.dumps({
    #'data': x
#})

#output = service.run(input_data)
#print(f"Predicted: {output}\nExpected: {y}")
#print(f"Output: {y}")


data = {
  "data": [
    {
                  'age': 75.0, 
                  'anaemia': 0.0, 
                  'creatinine_phosphokinase': 582.0,
                  'diabetes': 0.0,
                  'ejection_fraction': 20.0,
                  'high_blood_pressure': 1.0,
                  'platelets': 265000.0,
                  'serum_creatinine': 1.9,
                  'serum_sodium': 130.0,
                  'sex': 1.0,
                  'smoking': 0.0,
                  'time': 4.0
                 
                 
    }
 ]
}
# Convert to JSON string
input_data = json.dumps(data)
output = service.run(input_data)
print(f"Predicted: {output}\nExpected: {y}")

Predicted: [1]
Expected: 1


TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
#print the logs of the web service
print(service.get_logs())
#delete the service
service.delete()

2021-01-18T14:26:24,747068900+00:00 - iot-server/run 
2021-01-18T14:26:24,778447800+00:00 - gunicorn/run 
2021-01-18T14:26:24,773155700+00:00 - rsyslog/run 
2021-01-18T14:26:24,829129700+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd